In [1]:
# import json
# import logging
# import pychrome

# # Load your lookup tables
# with open("npcs_minified.json", encoding="utf-8") as f:
#     NPC_MAP = {n["_id"]: n["name"] for n in json.load(f)}
# with open("items.json", encoding="utf-8") as f:
#     ITEM_MAP = {i["id"]: i["name"] for i in json.load(f)}

# # Opcodes
# OP_HIT_EVENT  = 8
# OP_DROP_EVENT = 5

# # Set up file logging
# logging.basicConfig(
#     filename="combat.log",
#     filemode="a",
#     format="%(asctime)s %(message)s",
#     datefmt="%H:%M:%S",
#     level=logging.INFO
# )
# logger = logging.getLogger()


In [2]:
# # Cell 2 — attach to Chrome & listen for WebSocket frames (with raw logging)

# import datetime

# # 1. Connect to Chrome via CDP
# browser = pychrome.Browser(url="http://127.0.0.1:9222")
# tab = browser.list_tab()[0]
# tab.start()
# tab.Network.enable()
# tab.Page.enable()
# tab.Page.navigate(url="https://highspell.com/game")

# def handle_frame(**kwargs):
#     frame = kwargs.get("response", {})
#     payload = frame.get("payloadData", "")
#     timestamp = datetime.datetime.now().isoformat()

#     # 1) Raw log
#     with open("raw_ws.log", "a", encoding="utf-8") as rawf:
#         rawf.write(f"{timestamp} {payload}\n")

#     # 2) Decode only "42" frames
#     if not payload.startswith("42"):
#         return

#     _, batch = json.loads(payload[2:])
#     for op, pl in batch:
#         # Hit events
#         if op == OP_HIT_EVENT:
#             attacker, defender, dmg = pl
#             name_a = NPC_MAP.get(attacker, "Player")
#             name_d = NPC_MAP.get(defender,  "Player")
#             line = f"{timestamp} {name_a} hit {name_d}: {dmg}"

#         # Drop events
#         elif op == OP_DROP_EVENT:
#             _, item_id, qty, *_ = pl
#             item = ITEM_MAP.get(item_id, f"Item#{item_id}")
#             line = f"{timestamp} → Drop: {item} x{qty}"

#         else:
#             continue

#         # Print & decode‐log
#         print(line)
#         logger.info(line)

# # 3. Wire up WS frames
# tab.Network.webSocketFrameReceived = handle_frame

# print("Listening for WebSocket frames—raw payloads go to raw_ws.log, decoded events to combat.log.")


In [3]:
# import json
# import logging
# import datetime
# import pychrome

# # ─────── CONFIGURATION ──────────────────────────────────────────────────────────
# # Set to True to only log hits involving the IDs in PLAYER_FILTER
# PLAYER_FILTER_ENABLED = False
# PLAYER_FILTER = {"621523"}  # <-- your target player IDs

# # Map any player‐ids to friendly names; fall back to raw id when missing
# USER_MAP = {
#     "621523": "Answerth"
#     # "user456": "Bob",
#     # ... add more as you discover them
# }

# # Load your lookup tables
# with open("npcs_minified.json", encoding="utf-8") as f:
#     NPC_MAP = { n["_id"]: n["name"] for n in json.load(f) }

# with open("items.json", encoding="utf-8") as f:
#     ITEM_MAP = { i["id"]: i["name"] for i in json.load(f) }

# # Opcodes
# OP_HIT_EVENT  = 8
# OP_DROP_EVENT = 5

# # Set up file logging
# logging.basicConfig(
#     filename="combat.log",
#     filemode="a",
#     format="%(asctime)s %(message)s",
#     datefmt="%H:%M:%S",
#     level=logging.INFO
# )
# logger = logging.getLogger()


# # ─────── UTILITIES ──────────────────────────────────────────────────────────────
# def should_log_player(player_id: str) -> bool:
#     """Return True if we should include this player event."""
#     if not PLAYER_FILTER_ENABLED:
#         return True
#     return player_id in PLAYER_FILTER

# def resolve_user(player_id: str) -> str:
#     """Map id → username, or show raw id."""
#     return USER_MAP.get(player_id, f"Player/{player_id}")

# def format_hit_event(attacker: str, defender: str, dmg: int, ts: str) -> str:
#     """Produce the text line for a hit event."""
#     att_name = resolve_user(attacker) if attacker not in NPC_MAP else NPC_MAP[attacker]
#     def_name = resolve_user(defender) if defender not in NPC_MAP else NPC_MAP[defender]
#     return f"{ts} {att_name} hit {def_name}: {dmg}"

# def format_drop_event(item_id: str, qty: int, ts: str) -> str:
#     item = ITEM_MAP.get(item_id, f"Item#{item_id}")
#     return f"{ts} → Drop: {item} x{qty}"


# # ─────── FRAME HANDLER ───────────────────────────────────────────────────────────
# def handle_frame(**kwargs):
#     payload = kwargs.get("response", {}).get("payloadData", "")
#     ts = datetime.datetime.now().isoformat()

#     # Raw websocket dump
#     with open("raw_ws.log", "a", encoding="utf-8") as rawf:
#         rawf.write(f"{ts} {payload}\n")

#     if not payload.startswith("42"):
#         return

#     _, batch = json.loads(payload[2:])
#     for op, pl in batch:

#         # ─ Hit events ─────────────────────────────────────────────────────────
#         if op == OP_HIT_EVENT:
#             attacker, defender, dmg = pl

#             # filter out other players if needed
#             if PLAYER_FILTER_ENABLED and not (
#                 should_log_player(attacker) or should_log_player(defender)
#             ):
#                 continue

#             line = format_hit_event(attacker, defender, dmg, ts)

#         # ─ Drop events ────────────────────────────────────────────────────────
#         elif op == OP_DROP_EVENT:
#             _, item_id, qty, *_ = pl
#             line = format_drop_event(item_id, qty, ts)

#         else:
#             continue

#         # Output to console + log file
#         print(line)
#         logger.info(line)

#     # ── FUTURE PLACEHOLDERS ────────────────────────────────────────────────
#     # Here you could dispatch to other modules:
#     #   track_player_hp(...)
#     #   update_node_depletion(...)
#     # etc.


# # ─────── BROWSER SETUP ───────────────────────────────────────────────────────────
# browser = pychrome.Browser(url="http://127.0.0.1:9222")
# tab = browser.list_tab()[0]
# tab.start()
# tab.Network.enable()
# tab.Page.enable()
# tab.Page.navigate(url="https://highspell.com/game")

# tab.Network.webSocketFrameReceived = handle_frame
# print("Listening for WebSocket frames…")


# import time
# try:
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     tab.stop()
#     print("Stopped listening.")

In [4]:
# # Cell 1 — configuration, imports & maps

# # 1. Your current WebSocket session ID from the browser
# SESSION_ID = "t9ITJuz6CopZzndZAC_O"

# import json
# import socketio
# from threading import Thread

# # 2. Load your JSON definitions
# with open("npcs_minified.json", encoding="utf-8") as f:
#     NPC_MAP = {npc["_id"]: npc["name"] for npc in json.load(f)}

# with open("items.json", encoding="utf-8") as f:
#     ITEM_MAP = {item["id"]: item["name"] for item in json.load(f)}

# # 3. Opcode constants
# OP_HIT_EVENT     = 8    # [attackerId, defenderId, damage]
# OP_SKILL_XP      = 7    # [skillId, xpGained]
# OP_INVENTORY_UPD = 6    # [itemId, newCount]
# OP_HP_EVENT      = 13   # [entityId, hpOrXp]
# OP_DROP_EVENT    = 5    # [msgId, itemId, qty, ..., x, y]

# # 4. Known NPC IDs (from your JSON)
# BANDIT_ID  = 9
# PLAYER_ID  = None  # will auto-detect on first hit

# # 5. Initialize the Socket.IO client
# sio = socketio.Client(logger=False, reconnection=True)


In [5]:
# import json
# import logging
# import datetime
# import time
# import pychrome

# # ─────── CONFIGURATION ──────────────────────────────────────────────────────────
# RAW_LOG_PATH       = "raw_ws.log"
# COMBAT_LOG_PATH    = "combat.log"
# AMENDED_LOG_PATH   = "amended_raw.log"

# # Inventory specifics
# MAX_INVENTORY_SLOT = 27   # zero-based max slot index

# # Player filtering (currently disabled)
# PLAYER_FILTER_ENABLED = True
# PLAYER_FILTER         = {"621523", "665460"}

# USER_MAP = {
#     "621523": "Answerth",
#     "665460": "Answerth2"
# }

# CHAT_FILTERS = {
#     "global": {"enabled": True, "op_code": 90, "terms": []},
#     "local":  {"enabled": True, "op_code": 12, "terms": []},
# }

# # ─────── OPCODE CONSTANTS ───────────────────────────────────────────────────────
# OP_HIT_EVENT     = 8    # [attackerId, defenderId, damage]
# OP_SKILL_XP      = 7    # [skillId, xpGained]
# OP_INVENTORY_UPD = 6    # [itemId, newCount]
# OP_HP_EVENT      = 13   # [entityId, hpOrXp]
# OP_DROP_EVENT    = 5    # [msgId, itemId, qty, ..., x, y]

# # Load lookup tables
# with open("npcs_minified.json", encoding="utf-8") as f:
#     NPC_MAP = {n["_id"]: n["name"] for n in json.load(f)}

# with open("items.json", encoding="utf-8") as f:
#     ITEM_MAP = {i["id"]: i["name"] for i in json.load(f)}

# # Combat logger
# logging.basicConfig(
#     filename=COMBAT_LOG_PATH,
#     filemode="a",
#     format="%(asctime)s %(message)s",
#     datefmt="%H:%M:%S",
#     level=logging.INFO
# )
# logger = logging.getLogger()


# # ─────── UTILITIES ──────────────────────────────────────────────────────────────
# def should_log_player(player_id: str) -> bool:
#     return True if not PLAYER_FILTER_ENABLED else (player_id in PLAYER_FILTER)

# def resolve_user(player_id: str) -> str:
#     return USER_MAP.get(player_id, f"Player/{player_id}")

# def format_hit_event(attacker: str, defender: str, dmg: int, ts: str) -> str:
#     att_name = NPC_MAP.get(attacker) or resolve_user(attacker)
#     def_name = NPC_MAP.get(defender) or resolve_user(defender)
#     return f"{ts} {att_name} hit {def_name}: {dmg}"

# def format_drop_event(item_id: str, qty: int, ts: str) -> str:
#     item = ITEM_MAP.get(item_id, f"Item#{item_id}")
#     return f"{ts} → Drop: {item} x{qty}"


# # ─────── FRAME HANDLER ───────────────────────────────────────────────────────────
# def handle_frame(**kwargs):
#     payload = kwargs.get("response", {}).get("payloadData", "")
#     ts      = datetime.datetime.now().isoformat()

#     # 1) Raw dump
#     with open(RAW_LOG_PATH, "a", encoding="utf-8") as rawf:
#         rawf.write(f"{ts} {payload}\n")

#     # Prepare amended line
#     tag           = "[UNHANDLED]"
#     amended_line  = f"{ts} {payload}"

#     if payload.startswith("42"):
#         _, batch = json.loads(payload[2:])
#         for op, pl in batch:

#             # ── Combat events ─────────────────────────────────────────────
#             if op == OP_HIT_EVENT:
#                 attacker, defender, dmg = pl
#                 if PLAYER_FILTER_ENABLED and not (
#                     should_log_player(attacker) or should_log_player(defender)
#                 ):
#                     continue
#                 line = format_hit_event(attacker, defender, dmg, ts)
#                 tag  = "[TRACKED]"
#                 logger.info(line)

#             elif op == OP_DROP_EVENT:
#                 _, item_id, qty, *_ = pl
#                 line = format_drop_event(item_id, qty, ts)
#                 tag  = "[TRACKED]"
#                 logger.info(line)

#             # ── Global chat ──────────────────────────────────────────────
#             elif CHAT_FILTERS["global"]["enabled"] and op == CHAT_FILTERS["global"]["op_code"]:
#                 pid, username, msg = pl
#                 line = f"{ts} GLOBAL_CHAT {username}: {msg}"
#                 tag  = "[TRACKED]" if not CHAT_FILTERS["global"]["terms"] or any(t in msg for t in CHAT_FILTERS["global"]["terms"]) else "[IGNORED]"

#             # ── Local chat ───────────────────────────────────────────────
#             elif CHAT_FILTERS["local"]["enabled"] and op == CHAT_FILTERS["local"]["op_code"]:
#                 pid, _, msg = pl
#                 user = resolve_user(str(pid))
#                 line = f"{ts} LOCAL_CHAT {user}: {msg}"
#                 tag  = "[TRACKED]" if not CHAT_FILTERS["local"]["terms"] or any(t in msg for t in CHAT_FILTERS["local"]["terms"]) else "[IGNORED]"

#             # ── Item pickup ──────────────────────────────────────────────
#             elif op == 9:
#                 item_id, = pl
#                 name = ITEM_MAP.get(item_id, f"Item#{item_id}")
#                 line = f"{ts} PICKUP {name}"
#                 tag  = "[TRACKED]"

#             # ── State update (inventory, nodes, etc.) ────────────────────
#             elif op == 103:
#                 _, slot_idx, item_id, qty, *rest = pl
#                 item_name = ITEM_MAP.get(item_id, f"Item#{item_id}")
#                 if slot_idx >= MAX_INVENTORY_SLOT:
#                     line = f"{ts} INVENTORY FULL – could not place {item_name}"
#                 else:
#                     line = f"{ts} {item_name} placed in inventory slot {slot_idx1} x{qty}"
#                 tag  = "[TRACKED]"

#             else:
#                 continue

#             amended_line = f"{line} {tag}"
#             break

#     # 3) Append to amended log
#     with open(AMENDED_LOG_PATH, "a", encoding="utf-8") as amf:
#         amf.write(amended_line  "\n")

#     # 4) Echo amended to console
#     print(amended_line)


# # ─────── BROWSER SETUP & LOOP ───────────────────────────────────────────────────
# browser = pychrome.Browser(url="http://127.0.0.1:9222")
# tabs    = browser.list_tab()
# if not tabs:
#     raise RuntimeError("No debuggable tabs—did you start Chrome with --remote-debugging-port=9222?")
# tab = tabs[0]

# tab.start()
# tab.Network.enable()
# tab.Page.enable()
# tab.Page.navigate(url="https://highspell.com/game")
# tab.Network.webSocketFrameReceived = handle_frame

# print("✅ Listening for WebSocket frames. Press CtrlC to stop.")
# try:
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     tab.stop()
#     print("Stopped listening.")

# #Item#442 placed in inventory slot 20 x1 [TRACKED] is treasure map t1


In [6]:
#  & "C:\Program Files\Google\Chrome\Application\chrome.exe" `
# >>     --remote-debugging-port=9222 `
# >>     --user-data-dir="C:/tmp/chrome-debug" `
# >>     https://highspell.com/game

In [7]:
import json
import logging
import datetime
import time
import pychrome
from playsound import playsound

# ─────── PATHS & SLOTS ─────────────────────────────────────────────────────────
RAW_LOG_PATH       = "raw_ws.log"
COMBAT_LOG_PATH    = "combat.log"
AMENDED_LOG_PATH   = "amended_raw.log"
MAX_INVENTORY_SLOT = 27   # zero-based max slot index

# ─────── FLAGS ────────────────────────────────────────────────────────────────
SUPPRESS_CHAT   = True    # hide local chat prints
SUPPRESS_DAMAGE = False   # mute hit events when True

# ─────── DYNAMIC SELF IDENTIFICATION ───────────────────────────────────────────
MY_PLAYER_ID = None       # set only via “link_id” local chat
last_regen_minute = None

# ─────── OPCODE CONSTANTS ───────────────────────────────────────────────────────
OP_LOCAL_CHAT                = 12   # LocalChat
OP_INCREASE_COMBAT_EXP       = 7    # IncreaseCombatExp
OP_HIT_EVENT                 = 8    # ShowDamage
OP_OBTAINED_RESOURCE         = 9    # ObtainedResource (item pickup)
OP_DROP_EVENT                = 5    # ItemEnteredChunk (monster drop)
OP_STOPPED_SKILLING          = 36   # StoppedSkilling
OP_ACCURACY_UPDATE           = 39   # PlayerSkillLevelIncreased
OP_COMBAT_LEVEL_UPD          = 40   # PlayerCombatLevelIncreased
OP_PLAYER_DIED               = 50   # PlayerDied
OP_UPDATED_SHOP_STOCK        = 53   # UpdatedShopStock
OP_ENTITY_EXHAUSTED          = 62   # EntityExhaustedResources
OP_ENTITY_REPLENISHED        = 63   # EntityReplenishedResources
OP_GAINED_EXP                = 65   # GainedExp
OP_SKILL_LEVEL_CHANGED       = 79   # SkillCurrentLevelChanged
OP_HEALTH_RESTORED           = 91   # HealthRestored
OP_HP_UPDATE                 = 93   # ForcedSkillCurrentLevelChanged (true HP push)
OP_ADDED_ITEM_AT_INVENTORY   = 103  # AddedItemAtInventorySlot

current_hp = None   # will hold your latest known HP

  # ─────── ALERT CONFIG ────────────────────────────────────────────────────────

# drop triggers (case‐insensitive substrings) and untracked‐item prefix
DROP_ALERT_STRINGS = ["bass", "scroll", "logs", "brew", "milk", "rough", "celadium", "item"]
UNTRACKED_PREFIX   = "Item#"
ALERT_SOUND_FILE   = "sound.mp3"    # must exist in current dir

# ─────── LOAD LOOKUP TABLES ─────────────────────────────────────────────────────
with open("npcs_minified.json", encoding="utf-8") as f:
    NPC_MAP = {str(n["_id"]): n["name"] for n in json.load(f)}

with open("items.json", encoding="utf-8") as f:
    ITEM_MAP = {i["id"]: i["name"] for i in json.load(f)}

# ─────── COMBAT LOGGER SETUP ────────────────────────────────────────────────────
logging.basicConfig(
    filename=COMBAT_LOG_PATH,
    filemode="a",
    format="%(asctime)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO
)
logger = logging.getLogger()

# ─────── UTILITIES ──────────────────────────────────────────────────────────────
def resolve_user(pid: str) -> str:
    """Map your linked ID to “Me”, everything else to NPC name or generic."""
    if MY_PLAYER_ID is not None and str(pid) == str(MY_PLAYER_ID):
        return "Me"
    return NPC_MAP.get(str(pid), f"Player/{pid}")

# ─────── FORMATTERS ────────────────────────────────────────────────────────────
def format_hit_event(attacker: str, defender: str, dmg: int, ts: str) -> str:
    att_name = resolve_user(attacker)
    def_name = resolve_user(defender)
    return f"{ts} {att_name} hit {def_name}: {dmg}"

def format_drop_line(drops: list, ts: str) -> str:
    if len(drops) == 1:
        name, qty = drops[0]
        return f"{ts} → Drop: {name} x{qty}"
    parts = [f"{name} x{qty}" for name, qty in drops]
    return f"{ts} → Drops: " + ", ".join(parts)

# ─────── FRAME HANDLER ───────────────────────────────────────────────────────────
def handle_frame(**kwargs):
    global MY_PLAYER_ID, current_hp, last_regen_minute

    payload = kwargs.get("response", {}).get("payloadData", "")
    ts      = datetime.datetime.now().isoformat()
    now     = datetime.datetime.now()

    # ── Passive regen: +1 HP at approx :49 each minute ────────────────
    if current_hp is not None and now.second == 49:
        if last_regen_minute != now.minute:
            last_regen_minute = now.minute
            current_hp += 1
            regen_line = f"{ts} Me passively heal +1 → {current_hp}"
            logger.info(regen_line)
            print(f"{regen_line} [TRACKED]")

    # 1) Raw dump
    with open(RAW_LOG_PATH, "a", encoding="utf-8") as rawf:
        rawf.write(f"{ts} {payload}\n")

    amended_line = f"{ts} {payload}"
    tag = "[UNHANDLED]"

    if payload.startswith("42"):
        _, batch = json.loads(payload[2:])
        drop_events = []

        for op, pl in batch:
            # ── Collect drops ─────────────────────────────────────────
            if op == OP_DROP_EVENT:
                _, item_id, qty, *_ = pl
                name = ITEM_MAP.get(item_id, f"Item#{item_id}")
                drop_events.append((name, qty))
                continue

            # ── Hit events ───────────────────────────────────────────
            if op == OP_HIT_EVENT:
                if SUPPRESS_DAMAGE or MY_PLAYER_ID is None:
                    continue

                attacker, defender, dmg = pl
                # only care about hits where either party is you
                if str(attacker) != str(MY_PLAYER_ID) and str(defender) != str(MY_PLAYER_ID):
                    continue

                # subtract damage from your HP
                hp_note = ""
                if str(defender) == str(MY_PLAYER_ID) and current_hp is not None:
                    current_hp = max(0, current_hp - dmg)
                    hp_note = f" (HP → {current_hp})"

                line = format_hit_event(attacker, defender, dmg, ts) + hp_note
                tag  = "[TRACKED]"
                logger.info(line)
                print(f"{line} {tag}")

                # low-HP alert
                if current_hp is not None and current_hp < 20:
                    try:
                        playsound(ALERT_SOUND_FILE, block=False)
                    except Exception as e:
                        logger.error(f"Could not play sound: {e}")

                continue

            # ── True HP update (op 93) ─────────────────────────────────
            if op == OP_HP_UPDATE and MY_PLAYER_ID is not None:
                _, new_hp, _ = pl
                current_hp = new_hp
                line = f"{ts} Me’s HP set to {new_hp}"
                tag  = "[TRACKED]"
                logger.info(line)
                print(f"{line} {tag}")

                # low-HP alert
                if current_hp < 20:
                    try:
                        playsound(ALERT_SOUND_FILE, block=False)
                    except Exception as e:
                        logger.error(f"Could not play sound: {e}")

                continue

            # ── Accuracy update ────────────────────────────────────────
            if op == OP_ACCURACY_UPDATE:
                entity, *_, new_acc = pl
                name = resolve_user(entity)
                line = f"{ts} {name} accuracy is now {new_acc}"
                tag  = "[TRACKED]"
                logger.info(line)
                print(f"{line} {tag}")
                continue

            # ── Combat level update ───────────────────────────────────
            if op == OP_COMBAT_LEVEL_UPD:
                entity, new_lvl = pl
                name = resolve_user(entity)
                line = f"{ts} {name} combat level is now {new_lvl}"
                tag  = "[TRACKED]"
                logger.info(line)
                print(f"{line} {tag}")
                continue

            # ── Local chat (“link_id” handshake + optional echo) ──────────
            if op == OP_LOCAL_CHAT:
                pid, _, msg = pl

                # DEBUG: always dump the raw local-chat message
                print(f"{ts} [DEBUG] LOCAL_CHAT got: '{msg}' (pid={pid})")

                # Normalize and detect link_id
                normalized = msg.strip().lower()
                if normalized == "link_id":
                    MY_PLAYER_ID = pid
                    print(f"{ts} → Linked session player ID = {MY_PLAYER_ID}")

                # Now, if you still want to suppress other chat lines:
                if SUPPRESS_CHAT:
                    continue

                # Otherwise echo it like before
                user = resolve_user(pid)
                line = f"{ts} LOCAL_CHAT {user}: {msg}"
                tag  = "[TRACKED]"
                logger.info(line)
                print(f"{line} {tag}")
                continue

            # ── Inventory updates ──────────────────────────────────────
            # if op == OP_INVENTORY_UPD:
            #     container_id, slot_idx, item_id, qty, *rest = pl
            #     item_name = ITEM_MAP.get(item_id, f"Item#{item_id}")
            #     if container_id == 0:
            #         if slot_idx >= MAX_INVENTORY_SLOT:
            #             line = f"{ts} INVENTORY FULL – could not place {item_name}"
            #         else:
            #             line = f"{ts} {item_name} placed in inventory slot {slot_idx+1} x{qty}"
            #         tag  = "[TRACKED]"
            #         logger.info(line)
            #         print(f"{line} {tag}")
            #     continue

        # ── Emit grouped drops ───────────────────────────────────────
        if drop_events:
            line = format_drop_line(drop_events, ts)
            tag  = "[TRACKED]"
            logger.info(line)
            print(f"{line} {tag}")

            # drop-alert sounds
            for name, qty in drop_events:
                lname = name.lower()
                if any(sub in lname for sub in DROP_ALERT_STRINGS) or name.startswith(UNTRACKED_PREFIX):
                    try:
                        playsound(ALERT_SOUND_FILE, block=False)
                    except Exception as e:
                        logger.error(f"Could not play sound on drop alert: {e}")

    # 3) Append to amended log
    with open(AMENDED_LOG_PATH, "a", encoding="utf-8") as amf:
        amf.write(f"{line if 'line' in locals() else amended_line} {tag}\n")


# ─────── BROWSER SETUP & LOOP ───────────────────────────────────────────────────
browser = pychrome.Browser(url="http://127.0.0.1:9222")
tabs    = browser.list_tab()
if not tabs:
    raise RuntimeError("No debuggable tabs—did you start Chrome with --remote-debugging-port=9222?")

tab = tabs[0]
tab.start()
tab.Network.enable()
tab.Page.enable()
tab.Page.navigate(url="https://highspell.com/game")
tab.Network.webSocketFrameReceived = handle_frame

print("✅ Listening for WebSocket frames. Press Ctrl+C to stop.")
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    tab.stop()
    print("Stopped listening.")


✅ Listening for WebSocket frames. Press Ctrl+C to stop.
2025-05-10T00:45:47.903419 → Drops: giant bones x1, giant bones x1, potato x1, giant bones x1, coins x80, giant bones x1 [TRACKED]
2025-05-10T00:45:52.423473 → Drops: giant bones x1, potato x1 [TRACKED]
2025-05-10T00:46:06.219930 → Drop: giant bones x1 [TRACKED]
2025-05-10T00:46:14.629081 → Drop: bronze scimitar x1 [TRACKED]
2025-05-10T00:46:26.631908 → Drops: giant bones x1, giant bones x1 [TRACKED]
2025-05-10T00:46:34.469670 → Drops: giant bones x1, oak logs x3 [TRACKED]
2025-05-10T00:46:42.259354 [DEBUG] LOCAL_CHAT got: 'link_id' (pid=753256)
2025-05-10T00:46:42.259354 → Linked session player ID = 753256
2025-05-10T00:46:42.259354 Player/11962 hit Me: 0 [TRACKED]
2025-05-10T00:46:45.256987 Me hit Player/11962: 0 [TRACKED]
2025-05-10T00:46:45.856682 Player/11962 hit Me: 0 [TRACKED]
2025-05-10T00:46:48.257938 Me hit Player/11962: 1 [TRACKED]
2025-05-10T00:46:49.459192 Player/11962 hit Me: 0 [TRACKED]
2025-05-10T00:46:51.263262 Me

Exception in thread Thread-5 (_recv_loop):
Traceback (most recent call last):
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\site-packages\pychrome\tab.py", line 122, in _recv_loop


    message = json.loads(message_json)
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASpain\miniconda3\envs\hs_tailer\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
